# 1. Knihovna `multiprocessing`

`multiprocessing` nabízí API podobné `threading`, ale místo vláken spouští samostatné procesy. Každý proces má vlastní paměť i vlastní interpreter, takže se vyhne omezení GIL pro CPU-bound úlohy.

Další možnosti najdete v [oficiální dokumentaci](https://docs.python.org/3/library/multiprocessing.html).

## 1.1 Vytvoření procesu

Pro vytvoření procesu použijeme třídu `Process`.

Nejdůležitější parametry konstruktoru:
- `target`: funkce, která se má spustit,
- `args`/`kwargs`: argumenty funkce,
- `name`: název procesu.

V prostředí Jupyter notebooků na Pythonu 3.14 je praktické použít explicitně kontext `fork`, aby šly příklady s lokálními funkcemi spouštět přímo v buňkách.

In [ ]:
import multiprocessing

mp_ctx = multiprocessing.get_context("fork")


def square(x):
    print(x * x)


numbers = [1, 2, 3, 4, 5]
processes = []

for number in numbers:
    process = mp_ctx.Process(target=square, args=(number,))
    processes.append(process)
    process.start()

for process in processes:
    process.join()

## 1.2 Procesy vs. vlákna

- Procesy jsou oddělené instance programu s vlastní pamětí.
- U CPU-bound úloh mohou skutečně běžet současně na více jádrech.

In [ ]:
def vypocet():
    soucet = 0
    for i in range(30_000_000):
        soucet += i
    return soucet

In [ ]:
%time vypocet()

In [ ]:
import threading
import time

start = time.time()

vlakno1 = threading.Thread(target=vypocet)
vlakno2 = threading.Thread(target=vypocet)

vlakno1.start()
vlakno2.start()

vlakno1.join()
vlakno2.join()

konec = time.time()
print("Doba trvání:", konec - start)

In [ ]:
import multiprocessing
import time

mp_ctx = multiprocessing.get_context("fork")

start = time.time()

proces1 = mp_ctx.Process(target=vypocet)
proces2 = mp_ctx.Process(target=vypocet)

proces1.start()
proces2.start()

proces1.join()
proces2.join()

konec = time.time()
print("Doba trvání:", konec - start)

## 1.3 Sběr výstupů přes `Queue`

Vstupy můžeme předat přímo argumenty procesu. Pro bezpečný sběr výsledků z více procesů použijeme `Queue`.

In [ ]:
import multiprocessing

mp_ctx = multiprocessing.get_context("fork")


def square(x, output_queue):
    output_queue.put(x * x)


numbers = [1, 2, 3, 4, 5]
output_queue = mp_ctx.Queue()
processes = []

for number in numbers:
    process = mp_ctx.Process(target=square, args=(number, output_queue))
    processes.append(process)
    process.start()

for process in processes:
    process.join()

while not output_queue.empty():
    print(output_queue.get())

## 1.4 Komunikace přes `Pipe`

`Pipe` je obousměrný komunikační kanál mezi dvěma konci spojení.

In [ ]:
import multiprocessing

mp_ctx = multiprocessing.get_context("fork")


def square(x, conn):
    conn.send(x * x)
    conn.send(x * x * x)
    conn.close()


numbers = [1, 2, 3, 4, 5]
processes = []
parent_conns = []

for number in numbers:
    parent_conn, child_conn = mp_ctx.Pipe()
    parent_conns.append(parent_conn)
    process = mp_ctx.Process(target=square, args=(number, child_conn))
    processes.append(process)
    process.start()

for process in processes:
    process.join()

for parent_conn in parent_conns:
    print(parent_conn.recv())
    print(parent_conn.recv())
    parent_conn.close()

In [ ]:
import multiprocessing
import time

mp_ctx = multiprocessing.get_context("fork")


def child_process(conn):
    time.sleep(2)
    conn.send("Hello from child process!")
    conn.close()


parent_conn, child_conn = mp_ctx.Pipe()
process = mp_ctx.Process(target=child_process, args=(child_conn,))
process.start()

while True:
    if parent_conn.poll(0.5):
        try:
            message = parent_conn.recv()
            print(f"Received message: {message}")
        except EOFError:
            print("Spojení bylo ukončeno bez další zprávy.")
        break
    print("Zatím bez zprávy, čekám...")

process.join()
parent_conn.close()

Pozor: `poll()` může vrátit `True` i při uzavření spojení. Následné `recv()` pak vyhodí `EOFError`, pokud už není co číst.

## 1.5 Synchronizace procesů přes `Barrier`

In [ ]:
import multiprocessing
import time

mp_ctx = multiprocessing.get_context("fork")


def worker(barrier, worker_id):
    print(f"Worker {worker_id} - before barrier")
    time.sleep(1)
    barrier.wait()
    print(f"Worker {worker_id} - after barrier")


num_workers = 5
barrier = mp_ctx.Barrier(num_workers)
processes = [mp_ctx.Process(target=worker, args=(barrier, i)) for i in range(num_workers)]

for process in processes:
    process.start()

for process in processes:
    process.join()

## 1.6 Sdílená data (`Value`, `Array`)

In [ ]:
import multiprocessing
import time

mp_ctx = multiprocessing.get_context("fork")


def invert_array(ready_flag, shared_array):
    for i in range(len(shared_array)):
        shared_array[i] = -shared_array[i]
    ready_flag.value = 1


def print_array_when_ready(ready_flag, shared_array):
    while ready_flag.value == 0:
        time.sleep(0.1)
    print(shared_array[:])


ready = mp_ctx.Value('i', 0)
arr = mp_ctx.Array('i', range(10))

reader = mp_ctx.Process(target=print_array_when_ready, args=(ready, arr))
reader.start()

writer = mp_ctx.Process(target=invert_array, args=(ready, arr))
writer.start()

writer.join()
reader.join()

## 1.7 `Pool` procesů

`Pool` vytváří skupinu pracovních procesů a umí rozdělit nezávislé úlohy mezi ně. Typicky se používá s `map`.

In [ ]:
import multiprocessing
import os

mp_ctx = multiprocessing.get_context("fork")


def square(x):
    process_id = os.getpid()
    print(f"Process ID: {process_id} zpracovává číslo {x}")
    return x * x


numbers = [i for i in range(10)]
with mp_ctx.Pool(processes=4) as pool:
    results = pool.map(square, numbers)

print(results)